In [1]:
#from sklearnex import patch_sklearn
#patch_sklearn()
import math
#from langdetect import detect,detect_langs
#import validators
#import spacy
#from spacy import displacy
#import en_core_web_sm

import pandas as pd
import numpy as np
import os
import email
import email.policy
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import RegexpTokenizer,word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
#from gensim.parsing.preprocessing import STOPWORDS
#from gensim.models import word2vec
#from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
import heapq
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.feature_extraction.text import HashingVectorizer

import tensorflow as tf
from tensorflow import keras
from keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import gc

In [2]:
database = pd.read_csv("Database/dataBaseWithNER.csv")

database = database.drop(columns=["Unnamed: 0"])
database = database.dropna()
target = database["target"].values.tolist()
database

,email,target
0,start date hourahead timee cardinall hou...,0
1,service long desk price structure deal quote ...,0
2,start date cardinall hourahead timee card...,0
3,start date hourahead timee cardinall anc...,0
4,cardinall deliverable revenue management marke...,0
...,...,...
33340,bio matrix scientific group symbo bmxg p...,1
33341,cardinall step away hot naked webcam girl liv...,1
33342,need pill increase performance click seroius ...,1
33343,datee final nom inlet hpl eastrans car...,0


In [3]:
#wordCount = bagOfWord(emailsText)

#wordFrequency = heapq.nlargest(100, wordCount, wordCount.get)

#print(wordFrequency)
emailsText = []
for email in database["email"]:
    emailsText.append(email)

In [4]:
print(len(emailsText))

33341


# Representação vetorial GLOVE

In [5]:
categories = [0, 1]

X_train = emailsText[:26000]

Y_train = target[:26000]

X_test = emailsText[26000:-1]
Y_test = target[26000:-1]

a = 0

In [6]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
#text_ds = tf.data.Dataset.from_tensor_slices(emailsText).batch(128)
vectorizer.adapt(np.concatenate((X_train, X_test)), batch_size=128)
vocab = vectorizer.get_vocabulary()
print("Vocab : {}".format(vocab[:10]))
print("Vocab Size : {}".format(vectorizer.vocabulary_size()))
out = vectorizer(X_train[:5])
print("Output Shape : {}".format(out.shape))

Vocab : ['', '[UNK]', 'cardinall', 'datee', 'personn', 'orgg', 'moneyy', 'gpee', 'ect', 'company']
Vocab Size : 20000
Output Shape : (5, 200)


In [7]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'cardinall',
 'datee',
 'personn',
 'orgg',
 'moneyy',
 'gpee',
 'ect',
 'company',
 'com',
 'timee',
 'e',
 'percentt',
 'time',
 'price',
 'business',
 'information',
 'said',
 'hou',
 'market',
 'new',
 'gas',
 'http',
 'email',
 'message',
 'need',
 'enron',
 'energy',
 'stock',
 'deal',
 'know',
 'mail',
 'cc',
 'service',
 'report',
 'thanks',
 'like',
 'power',
 'statement',
 'norpp',
 'number',
 'www',
 'security',
 'share',
 'c',
 'risk',
 'vince',
 'product',
 'investment',
 'group',
 'trading',
 'j',
 'ordinall',
 'money',
 'work',
 'forward',
 'let',
 'p',
 'contact',
 'want',
 'order',
 'term',
 'financial',
 'free',
 'sent',
 'credit',
 'offer',
 'question',
 'address',
 'use',
 'change',
 'list',
 'best',
 'management',
 'project',
 'right',
 'account',
 'based',
 'future',
 'customer',
 'meeting',
 'original',
 'site',
 'people',
 'r',
 'transaction',
 'date',
 'investor',
 'sale',
 'program',
 'contract',
 'office',
 'click',
 'data',
 'news',
 'help',


In [8]:
def create_model():
    return Sequential([
        layers.Input(shape=(1,), dtype="string"),
        vectorizer,
        #layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(len(categories), activation="softmax"),
    ])

In [9]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               25728     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 34,114
Trainable params: 34,114
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [11]:
history = model.fit(X_train, Y_train, batch_size=256, epochs=8, validation_data=(X_test, Y_test))
gc.collect()

Epoch 1/8
102/102 [==============================] - 4s 26ms/step - loss: 131.0792 - accuracy: 0.5271 - val_loss: 30.5798 - val_accuracy: 0.5286
Epoch 2/8
102/102 [==============================] - 1s 14ms/step - loss: 10.4005 - accuracy: 0.5424 - val_loss: 2.5667 - val_accuracy: 0.5486
Epoch 3/8
102/102 [==============================] - 1s 13ms/step - loss: 1.4121 - accuracy: 0.5498 - val_loss: 1.4258 - val_accuracy: 0.5403
Epoch 4/8
102/102 [==============================] - 1s 12ms/step - loss: 0.9584 - accuracy: 0.5574 - val_loss: 1.1114 - val_accuracy: 0.5565
Epoch 5/8
102/102 [==============================] - 1s 12ms/step - loss: 0.8210 - accuracy: 0.5622 - val_loss: 1.0459 - val_accuracy: 0.5658
Epoch 6/8
102/102 [==============================] - 1s 12ms/step - loss: 0.7596 - accuracy: 0.5667 - val_loss: 0.9868 - val_accuracy: 0.5650
Epoch 7/8
102/102 [==============================] - 1s 12ms/step - loss: 0.7351 - accuracy: 0.5700 - val_loss: 1.0002 - val_accuracy: 0.5651
Ep

6674

In [13]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
#print("\nClassification Report : ")
#print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=categories))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_test, np.argmax(test_preds, axis=1)))

Train Accuracy : 0.5299615384615385
Test  Accuracy : 0.5186648501362398

Confusion Matrix : 
[[3173  459]
 [3074  634]]


In [16]:
import scikitplot as skplt
import matplotlib.pyplot as plt

skplt.metrics.plot_confusion_matrix([categories[i] for i in Y_test], [categories[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=True,
                                    title="Confusion Matrix",
                                    cmap="Purples",
                                    hide_zeros=True,
                                    figsize=(5,5)
                                    );
plt.xticks(rotation=90);

ModuleNotFoundError: No module named 'scikitplot'

In [ ]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
word_index

In [ ]:
test = ["cardinall", "datee", "personn", "orgg", "moneyy", "gpee"]
[word_index[w] for w in test]

In [ ]:
model = TSNE(n_components=2, random_state=0)
#model = PCA(n_components=50, svd_solver='full')
array_red = model.fit_transform(dfTfidfArray)

df_tsne = pd.DataFrame(array_red)

target = dfTfidfArray["Target"].array

df_tsne['Target'] = target
print(df_tsne)
df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]

df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]

plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')

plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')

plt.title('Dados')
plt.xlabel('x')
plt.ylabel('y')

plt.show()

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(df_features.values,target,test_size=0.2)
modelo = getModel().fit(X_treino,y_treino)
score = modelo.score(X_teste,y_teste)
#score = modelo.score(["alo"],[1])
score

In [ ]:
scores = cross_val_score(getModel(),df_features.values,target,cv=10)

scores.mean()

In [ ]:
predicoes = cross_val_predict(getModel(), df_features.values, target, cv=10)

cm = confusion_matrix(target,predicoes,labels=[0, 1])

cm_df = pd.DataFrame(cm, columns=[0, 1])

cm_df

In [ ]:
emailsTest2 = []

for email in teste2["text"]:
    emailsTest2.append(wordsPreProcessing(email))
    
print("ok")

In [ ]:
emailsTest3 = []

for email in teste["text"]:
    emailsTest3.append(wordsPreProcessing(email))
    
print("ok")

In [ ]:
#APAGAR DEPOIS

# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(emailsTest)
# X_train_counts.shape
#
# tf_transformer = TfidfTransformer()
# X_train_tfidf = tf_transformer.fit_transform(X_train_counts)
# X_train_tfidf.shape

In [ ]:
# ##APAGAR DEPOIS
# clf = getModel().fit(X_train_tfidf,target)
#
# X_new_counts = count_vect.transform(emailsTest2)
#
# X_new_tfidf = tf_transformer.transform(X_new_counts)
#
# predict = clf.predict(X_new_tfidf)
# print(len(emailsTest2))
# print(predict)
# print(targetTest2)
# print(teste2)
# print(clf.score(X_new_tfidf,targetTest2))
#
# #predicoesTest = cross_val_predict(modelo2, dfTfidfArrayFeaturesTest.values, dfTfidfArrayTargetTest, cv=10)
#
# cmTest = confusion_matrix(targetTest2,predict,labels=[0, 1])
#
# cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])
#
# cm_dfTest

In [ ]:
# dfTfidfArrayTest = pd.DataFrame(data=tfidfTransformTest.toarray(), columns=tfidfVectorizerTest.get_feature_names_out())
# dfTfidfArrayTest.insert(len(dfTfidfArrayTest.columns), "Target", target.array, True)
# dfTfidfArrayTest = dfTfidfArrayTest.sample(frac=1)
# dfTfidfArrayTest


In [ ]:
# modelTest = TSNE(n_components=2, random_state=0)
# #modelTest = PCA(n_components=50)
# array_redTest = modelTest.fit_transform(dfTfidfArrayTest)
#
# df_tsneTest = pd.DataFrame(array_redTest)
# df_tsneTest['Target'] = target.array
#
# df_tsne_c1Test = df_tsneTest[df_tsneTest['Target'] == 0]
#
# df_tsne_c2Test  = df_tsneTest[df_tsneTest['Target'] == 1]
#
# plt.scatter(df_tsne_c1Test[0].array,df_tsne_c1Test[1].array,marker='o',color='blue')
#
# plt.scatter(df_tsne_c2Test[0].array,df_tsne_c2Test[1].array,marker='o',color='red')
#
# plt.title('Dados')
# plt.xlabel('x')
# plt.ylabel('y')
#
# plt.show()


In [ ]:
# dfTfidfArrayTarget = dfTfidfArray["Target"].array
# dfTfidfArrayFeatures = dfTfidfArray.drop(columns=['Target'])
#
# print(dfTfidfArrayFeatures)
#
# dfTfidfArrayTargetTest = dfTfidfArrayTest["Target"].array
# dfTfidfArrayFeaturesTest = dfTfidfArrayTest.drop(columns=['Target'])
#
#
#
# print(dfTfidfArrayFeaturesTest)
# dfTfidfArrayTargetTest

In [ ]:
# dfTfidfArrayTargetTest2 = dfTfidfArrayTest2["Target"].array
# dfTfidfArrayFeaturesTest2 = dfTfidfArrayTest2.drop(columns=['Target'])

In [ ]:
# #from sklearn.svm import LinearSVC
#
# X_treino, X_teste, y_treino, y_teste = train_test_split(dfTfidfArrayFeaturesTest2.values,dfTfidfArrayTargetTest2,test_size=0.2)
# X_treinoTest, X_testeTest, y_treinoTest, y_testeTest = train_test_split(dfTfidfArrayFeaturesTest.values,dfTfidfArrayTargetTest,test_size=0.01)
# modelo = getModel().fit(X_treino,y_treino)
# modelo2 = getModel().fit(X_treinoTest,y_treinoTest)
# predict = modelo.predict(X_teste)
# #score = modelo.score(X_treinoTest,y_treinoTest)
# #score2 = modelo2.score(X_treino,y_treino)
#
# cmTest = confusion_matrix(y_teste,predict)
# cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])
#
# cm_dfTest

In [ ]:
#predicoesTest = cross_val_predict(modelo2, dfTfidfArrayFeaturesTest.values, dfTfidfArrayTargetTest, cv=10)

#cmTest = confusion_matrix(dfTfidfArrayTargetTest,predicoesTest,labels=[0, 1])

#cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])

#cm_dfTest